<h3> BT4222 Project - Toxic Comments Classifier (Bot Application) </h3>

In this notebook, we will develop the Telegram bot which is integrated with our machine learning model. For the demo on the bot please refer to the report.

In [1]:
import pandas as pd
from collections import defaultdict
import io
import re
from nltk.corpus import stopwords
import string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score
import requests  
import datetime
import urllib.request
import bs4
import random
import re
import pickle
import warnings
import logging
import copy
import os
import glob
# import newspaper
import numpy as np
import time

In [ ]:
train = pd.read_csv('clean_train.csv').iloc[1:, 1:]
test = pd.read_csv('clean_train.csv').iloc[:, 1:].loc[[0], :]

In [ ]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [ ]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
test_term_doc = vec.transform(test[COMMENT])

In [ ]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [ ]:
x = trn_term_doc
test_x = test_term_doc

In [ ]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [ ]:
label_cols = ['toxic', 'severe toxic', 'obscene', 'threat', 'insult', 'identity hate']

In [ ]:
preds = np.zeros((len(test), len(label_cols)))

In [ ]:
d_model = dict()
for i, j in enumerate(label_cols):
    print('fit', j)
    d_model[i] , r = get_mdl(train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

In [5]:
'ABC'.lower()

'abc'

In [4]:
'abc' in 'hello babc'

True

In [19]:
# Replace with your own telegram bot token
token = ""


class BotHandler:

    def __init__(self, token):
        self.token = token
        self.dic = dict()
        self.api_url = "https://api.telegram.org/bot{}/".format(token)

    def get_updates(self, offset=None, timeout=3):
        method = 'getUpdates'
        params = {'timeout': timeout, 'offset': offset}
        resp = requests.get(self.api_url + method, params)
        result_json = resp.json()['result']
        return result_json

    def send_message(self, chat_id, text):
        params = {'chat_id': chat_id, 'text': text, 'parse_mode':'Markdown'}
        method = 'sendMessage'
        resp = requests.post(self.api_url + method, params)
        return resp
    
    def delete_message(self, chat_id, message_id):
        params = {'chat_id': chat_id, 'message_id':message_id}
        method = 'deleteMessage'
        resp = requests.post(self.api_url + method, params)
        return resp
    
    def kick_user(self, chat_id, userid):
        params = {'chat_id': chat_id, 'user_id':userid}
        method = 'kickChatMember'
        resp = requests.post(self.api_url + method, params)
        return resp
    
    def get_last_update(self):
        get_result = self.get_updates()
        last_update = None
        last_update = get_result[-1]
        return last_update

    def receive_text(self, text, chat_id, message_id, chat_user, userid):
#         pred_term_doc = vec.transform(pd.Series(text))
#         pred_x = pred_term_doc
#         preds = np.zeros((1, len(label_cols)))
#         for i, j in enumerate(label_cols):
#             preds[:,i] = d_model[i].predict_proba(pred_x.multiply(r))[:,1]
#             preds = np.round(preds)
        forbidden_words = ['gervaise','gervy','gheezy','gervais', 'gheezzyvaise','gheezzy']
        checker = False
        for word in forbidden_words:
            if word in text.lower():
                checker = True
                break
        if checker:
#             text = ""
#             for i in range(len(preds[0])):
#                 if list(preds[0])[i] == 1:
#                     if len(text) == 0:
#                         text = label_cols[i]
#                     else:
#                         text += ", {}".format(label_cols[i])
            text = "Message has been deleted. Due to Mention of Forbidden Word"
            bot.send_message(chat_id, text)
            bot.delete_message(chat_id, message_id)
            if userid in self.dic:
                self.dic[userid] += 1
            else:
                self.dic[userid] = 1
            valu = self.dic[userid]
            bot.send_message(chat_id, "Warning {}/3 given to {}!".format(valu, chat_user))
            if valu >= 3:
                bot.send_message(chat_id, "Last chance has been given! {} has been banned.".format(chat_user))
                bot.kick_user(chat_id, userid)

        
                
                
            
bot = BotHandler(token)  
greetings = ('hello', 'hi', 'greetings', 'test')


def main():
    new_offset = bot.get_updates()[-2]['update_id']
    while True:
        print("Polling")
        try:
            last_update = bot.get_last_update()
            last_update_id = last_update['update_id']

            if last_update_id != new_offset:
                print(last_update)
                last_chat_text = last_update['message']['text']
                last_message_id = last_update['message']['message_id']
                last_chat_id = last_update['message']['chat']['id']
                last_chat_name = last_update['message']['from']['first_name']
                last_chat_userid = last_update['message']['from']['id']
                bot.receive_text(last_chat_text, last_chat_id,last_message_id, last_chat_name, last_chat_userid)
                new_offset = last_update_id
        except UnicodeEncodeError:
            print("Unicode detected")
        time.sleep(1)
main()

Polling
{'update_id': 551804921, 'message': {'message_id': 10117, 'from': {'id': 102448232, 'is_bot': False, 'first_name': 'Russell', 'last_name': 'Ong', 'username': 'Hayimrussell'}, 'chat': {'id': -1001389758433, 'title': 'NOC Team + SG Manager: Jem', 'type': 'supergroup'}, 'date': 1586417198, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}}
Polling
Polling
Polling
Polling
Polling
Polling
Polling
Polling
Polling
Polling
Polling
{'update_id': 551804922, 'message': {'message_id': 10118, 'from': {'id': 22730456, 'is_bot': False, 'first_name': 'Jeremy', 'username': 'Jem_Lim', 'language_code': 'en'}, 'chat': {'id': -1001389758433, 'title': 'NOC Team + SG Manager: Jem', 'type': 'supergroup'}, 'date': 1586417508, 'text': 'Gheezy'}}
Polling
Polling
{'update_id': 551804923, 'message': {'message_id': 10121, 'from': {'id': 22730456, 'is_bot': False, 'first_name': 'Jeremy', 'username': 'Jem_Lim', 'language_code': 'en'}, 'chat': {'id': -1001389758433, 'title': 'N

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [7]:
bot = BotHandler(token)  

In [13]:
bot.get_updates()

[{'update_id': 661916489,
  'message': {'message_id': 1,
   'from': {'id': 22730456,
    'is_bot': False,
    'first_name': 'Jeremy',
    'username': 'Jem_Lim',
    'language_code': 'en'},
   'chat': {'id': 22730456,
    'first_name': 'Jeremy',
    'username': 'Jem_Lim',
    'type': 'private'},
   'date': 1586410964,
   'text': '/start',
   'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}},
 {'update_id': 661916490,
  'message': {'message_id': 2,
   'from': {'id': 22730456,
    'is_bot': False,
    'first_name': 'Jeremy',
    'username': 'Jem_Lim',
    'language_code': 'en'},
   'chat': {'id': 22730456,
    'first_name': 'Jeremy',
    'username': 'Jem_Lim',
    'type': 'private'},
   'date': 1586410968,
   'text': 'Test'}},
 {'update_id': 661916491,
  'message': {'message_id': 3,
   'from': {'id': 22730456,
    'is_bot': False,
    'first_name': 'Jeremy',
    'username': 'Jem_Lim',
    'language_code': 'en'},
   'chat': {'id': 22730456,
    'first_name': 'Jeremy',
    '